# Preprocess and make dataset

In [ ]:
# pip install emoji

In [ ]:
# pip install spacy

In [ ]:
# pip install langdetect

In [ ]:
# pip install --upgrade fasttext

In [ ]:
# !python -m spacy download en_core_web_sm

In [ ]:
import pandas as pd
import re
import emoji
from keras.preprocessing.text import text_to_word_sequence
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
import fasttext

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords

In [ ]:
# nltk.download('averaged_perceptron_tagger')

## Dataset

In [ ]:
fifa = 'tweets/fifa_world_cup_2022_tweets.csv'
g_of_t = 'tweets/GameofThronesS8.csv'
trump = 'tweets/hashtag_donaldtrump.csv'
biden = 'tweets/hashtag_joebiden.csv'

### FIFA

In [ ]:
# read as dataframe
fifa = pd.read_csv(fifa)
fifa.head()

,Unnamed: 0,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
0,0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadB...,neutral
1,1,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
2,2,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 htt...,positive
3,3,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/...,positive
4,4,2022-11-20 23:58:28+00:00,0,Twitter for Android,"If the BBC cares so much about human rights, h...",negative


#### Make df columns to be ['Tweet', 'Topics', 'Human_generated']

In [ ]:
fifa_df = fifa[['Tweet']].copy()
fifa_df.loc[:, 'Topics'] = 'Fifa'
fifa_df.loc[:, 'Human_generated'] = 1

fifa_df.head()

,Tweet,Topics,Human_generated
0,What are we drinking today @TucanTribe \n@MadB...,Fifa,1
1,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,Fifa,1
2,Worth reading while watching #WorldCup2022 htt...,Fifa,1
3,Golden Maknae shinning bright\n\nhttps://t.co/...,Fifa,1
4,"If the BBC cares so much about human rights, h...",Fifa,1


In [ ]:
fifa_df.isnull().sum()

Tweet              0
Topics             0
Human_generated    0
dtype: int64

### Game of ThronesS8

In [ ]:
game = pd.read_csv(g_of_t)

/var/folders/3y/45kdydws3r36_58_j4drldhm0000gn/T/ipykernel_5784/3428197056.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  game = pd.read_csv(g_of_t)


In [ ]:
game.columns

Index(['user_id', 'status_id', 'created_at', 'screen_name', 'text', 'source',
       'display_text_width', 'reply_to_status_id', 'reply_to_user_id',
       'reply_to_screen_name', 'is_quote', 'is_retweet', 'favorite_count',
       'retweet_count', 'hashtags', 'symbols', 'urls_url', 'urls_t.co',
       'urls_expanded_url', 'media_url', 'media_t.co', 'media_expanded_url',
       'media_type', 'ext_media_url', 'ext_media_t.co',
       'ext_media_expanded_url', 'ext_media_type', 'mentions_user_id',
       'mentions_screen_name', 'lang', 'quoted_status_id', 'quoted_text',
       'quoted_created_at', 'quoted_source', 'quoted_favorite_count',
       'quoted_retweet_count', 'quoted_user_id', 'quoted_screen_name',
       'quoted_name', 'quoted_followers_count', 'quoted_friends_count',
       'quoted_statuses_count', 'quoted_location', 'quoted_description',
       'quoted_verified', 'retweet_status_id', 'retweet_text',
       'retweet_created_at', 'retweet_source', 'retweet_favorite_count',
    

#### Make df columns to be ['Tweet', 'Topics', 'Human_generated']¶

In [ ]:
game_df = game[['text']].copy()
game_df.rename(columns = {'text': 'Tweet'}, inplace=True)
game_df.loc[:, 'Topics'] = 'Game'
game_df.loc[:, 'Human_generated'] = 1
game_df.head()

,Tweet,Topics,Human_generated
0,👍 on @YouTube: GAME OF THRONES 8x01 Breakdown!...,Game,1
1,👍 on @YouTube: Ups and Downs From Game Of Thro...,Game,1
2,Liked on YouTube: Ups and Downs From Game Of T...,Game,1
3,Liked on YouTube: GAME OF THRONES 8x01 Breakdo...,Game,1
4,@MrLegenDarius unpopular opinion: game of thro...,Game,1


In [ ]:
game_df.isnull().sum()

Tweet              0
Topics             0
Human_generated    0
dtype: int64

### Elections

In [ ]:
trump = pd.read_csv(trump, lineterminator='\n')
trump.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,3.316176e+08,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
3,2020-10-15 00:00:02,1.316529e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,8.283556e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591
4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121


In [ ]:
trump_df = trump[['tweet']].copy()
trump_df.rename(columns = {'tweet': 'Tweet'}, inplace=True)
trump_df.loc[:, 'Topics'] = 'Election'
trump_df.loc[:, 'Human_generated'] = 1
trump_df.head()

,Tweet,Topics,Human_generated
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,Election,1
1,"Usa 2020, Trump contro Facebook e Twitter: cop...",Election,1
2,"#Trump: As a student I used to hear for years,...",Election,1
3,2 hours since last tweet from #Trump! Maybe he...,Election,1
4,You get a tie! And you get a tie! #Trump ‘s ra...,Election,1


In [ ]:
trump_df.isna().sum()

Tweet              0
Topics             0
Human_generated    0
dtype: int64

In [ ]:
trump_df.shape

(970919, 3)

In [ ]:
biden = pd.read_csv(biden, lineterminator='\n')
biden.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.316529e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,8.099044e+08,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.316529e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3.494182e+09,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.316529e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032807e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132


In [ ]:
biden_df = biden[['tweet']].copy()
biden_df.rename(columns = {'tweet': 'Tweet'}, inplace=True)
biden_df.loc[:, 'Topics'] = 'Election'
biden_df.loc[:, 'Human_generated'] = 1
biden_df.head()

,Tweet,Topics,Human_generated
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,Election,1
1,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,Election,1
2,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,Election,1
3,@chrislongview Watching and setting dvr. Let’s...,Election,1
4,#censorship #HunterBiden #Biden #BidenEmails #...,Election,1


In [ ]:
biden_df.isna().sum()

Tweet              0
Topics             0
Human_generated    0
dtype: int64

In [ ]:
# Combine Trump data and Biden data to be election data
election_df = pd.concat([trump_df, biden_df], axis=0, ignore_index=True)

In [ ]:
election_df.isna().sum()

Tweet              0
Topics             0
Human_generated    0
dtype: int64

In [ ]:
print(election_df.shape, biden_df.shape[0] + trump_df.shape[0], biden_df.shape[0] + trump_df.shape[0] == election_df.shape)

(1747805, 3) 1747805 False


### check three dataset

In [ ]:
election_df.head()

,Tweet,Topics,Human_generated
0,#Elecciones2020 | En #Florida: #JoeBiden dice ...,Election,1
1,"Usa 2020, Trump contro Facebook e Twitter: cop...",Election,1
2,"#Trump: As a student I used to hear for years,...",Election,1
3,2 hours since last tweet from #Trump! Maybe he...,Election,1
4,You get a tie! And you get a tie! #Trump ‘s ra...,Election,1


In [ ]:
election_df.shape

(1747805, 3)

In [ ]:
game_df.head()

,Tweet,Topics,Human_generated
0,👍 on @YouTube: GAME OF THRONES 8x01 Breakdown!...,Game,1
1,👍 on @YouTube: Ups and Downs From Game Of Thro...,Game,1
2,Liked on YouTube: Ups and Downs From Game Of T...,Game,1
3,Liked on YouTube: GAME OF THRONES 8x01 Breakdo...,Game,1
4,@MrLegenDarius unpopular opinion: game of thro...,Game,1


In [ ]:
game_df.shape

(760660, 3)

In [ ]:
fifa_df.head()

,Tweet,Topics,Human_generated
0,What are we drinking today @TucanTribe \n@MadB...,Fifa,1
1,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,Fifa,1
2,Worth reading while watching #WorldCup2022 htt...,Fifa,1
3,Golden Maknae shinning bright\n\nhttps://t.co/...,Fifa,1
4,"If the BBC cares so much about human rights, h...",Fifa,1


In [ ]:
fifa_df.shape

(22524, 3)

# Text Preprocessing

- Remove punctuation, @XXX, emoji, <> to become clean text
- Consider all https as URL
- Count @XXX, emoji

- Remove stopwords
- lemmatizer
- tokenize

- Sorted clean text using len(token) in descending order

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(' ', text)

# Test
# text_with_emojis = "Hello 😊, how are you? 🌟"
# text_without_emojis = remove_emojis(text_with_emojis)
# print(text_without_emojis)  # Output: Hello , how are you?

In [ ]:
# make a text preprocessing function

def text_preprocessing_part1(df):
    """
    return None. Modify in place.
    """
    # =======Removeing=======

    df['Tweet'] = df['Tweet'].apply(lambda x: re.sub('(<.*?>)', '', x))

    # remove punctuation
    df['Tweet'] = df['Tweet'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))

    # remove larger than
    df['Tweet'] = df['Tweet'].str.replace(r'&[gl]t;', '', regex=True)

    # counting @
    df['count_profile'] = df['Tweet'].apply(lambda x: len(re.findall(r'@\w+', x)))

    # Counting emoji
    df['count_emoji'] = df['Tweet'].apply(lambda x: emoji.emoji_count(x))

    # clean tweet removing @ and emoji
    df['clean_tweet'] = df['Tweet'].apply(lambda x: re.sub(r'@\w+', '', remove_emojis(x)))

    # remove whitespace
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: x.strip())

    # temp clean_tweet
    # Define a regular expression to match URLs
    url_pattern = r'https\S+'
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(url_pattern, 'URL', x))

    # only charac
#     df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('[^a-zA-Z"]', ' ', x))
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('[^a-zA-Z0-9"]', ' ', x))

    # convert to lower
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: x.lower())


    # =======count length=======
    df['Tweet_Length'] = df['Tweet'].apply(len)
    df['clean_tweet_Length'] = df['clean_tweet'].apply(len)

    # =======Reorder columns=======
    # Specify the new order of columns
    new_order = ['clean_tweet', 'Topics', 'Human_generated', 'count_emoji', 'count_profile', 'clean_tweet_Length', 'Tweet_Length', 'Tweet']

    # Change the order of columns and
    df = df.reindex(columns=new_order)

    # reset index and return a new copy of dataframe
    df = df.reset_index().iloc[:, 1:].copy()

    # =======Sort and reIndex=======
    ### not doing sort here, because we should sort by count of words. # REMEMBER to deal with https
    # sort by len(clean_tweet)
    # df.sort_values(by='clean_tweet', key=lambda x: x.str.len(), ascending=False, inplace=True)

    return df

In [ ]:
def text_preprocessing_part2(df):
#     # Define a function to map POS tags from Treebank to WordNet
#     def get_wordnet_pos(treebank_tag):
#         if treebank_tag.startswith('J'):
#             return wordnet.ADJ
#         elif treebank_tag.startswith('V'):
#             return wordnet.VERB
#         elif treebank_tag.startswith('N'):
#             return wordnet.NOUN
#         elif treebank_tag.startswith('R'):
#             return wordnet.ADV
#         else:
#             return wordnet.NOUN  # default to noun if POS tag not found

#     def lemmatize_word(word):
#         pos_tag = nltk.pos_tag([word])[0][1]  # Get POS tag of the word
#         pos = get_wordnet_pos(pos_tag)  # Map POS tag to WordNet POS
#         return lemmatizer.lemmatize(word, pos=pos)  # Lemmatize the word
    # =========not use above===can't catch: catch, saw... =====================================================================================

    # Load the English language model
    nlp = spacy.load("en_core_web_sm")

    # Define a function to lemmatize a text
    def lemmatize_text(text):
        doc = nlp(text)
        lemmatized_text = " ".join([token.lemma_ for token in doc])
        return lemmatized_text

    # =======Preprocessing start========================================================================================
    # tokenize
    df['tokenize_clean_tweet'] = df['clean_tweet'].apply(lambda x: text_to_word_sequence(x))

    # remove stop words
#     df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [w for w in x
#                                                                              if w not in stop_words and
#                                                                              not w in ENGLISH_STOP_WORDS and
#                                                                              not w in gensim_stopwords])
    # lemmatizer: to its original form
    df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [lemmatize_text(token) for token in x])

    # newly add: because "gonna" -> "go to"
    df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [word for w in x for word in text_to_word_sequence(w)])


    # remove stop words
    df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [w for w in x
                                                                             if w not in stop_words and
                                                                             w not in ENGLISH_STOP_WORDS and
                                                                             w not in gensim_stopwords])
    # transfer back to string clean_tweet
    df['clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: " ".join(x))

    # sort by token_length
    df['token_length'] = df['tokenize_clean_tweet'].apply(len)

    df.sort_values(by='token_length', ascending=False, inplace=True)



In [ ]:
def text_preprocessing(ori_df):
    df = text_preprocessing_part1(ori_df)
    text_preprocessing_part2(df)
    return df

In [ ]:
# example
df = {'Tweet': ["new, let, not to go", "worked we has him one us day good would apples caught", "he final the his is us we", "end, still first bad"]}
df = pd.DataFrame(df)
df
text_preprocessing(df)

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
1,work day good apple catch,NaN,NaN,0,0,53,53,worked we has him one us day good would apples...,"[work, day, good, apple, catch]",5
0,new let,NaN,NaN,0,0,17,17,new let not to go,"[new, let]",2
3,end bad,NaN,NaN,0,0,19,19,end still first bad,"[end, bad]",2
2,final,NaN,NaN,0,0,25,25,he final the his is us we,[final],1


In [ ]:
fifa_df = text_preprocessing(fifa_df)
fifa_df.head()

NameError: name 'fifa_df' is not defined

In [ ]:
game_df = text_preprocessing(game_df)
game_df.head()

NameError: name 'game_df' is not defined

In [ ]:
election_df = text_preprocessing(election_df)
election_df.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
1379704,2 7 0 2 7 0 2 7 0 2 7 0 2 7 0 2 7 0 2 7 0 2 7 ...,Election,1,0,0,265,265,2 7 0 - 2 7 0 -2 7 0 - 2 7 0 - 2 7 0 -2 7 0 - ...,"[2, 7, 0, 2, 7, 0, 2, 7, 0, 2, 7, 0, 2, 7, 0, ...",98
1394189,wi mi wi mi wi mi wi mi wi mi wi mi wi mi wi m...,Election,1,3,0,271,276,WI MI WI MI WI MI WI MI WI MI WI MI WI MI WI M...,"[wi, mi, wi, mi, wi, mi, wi, mi, wi, mi, wi, m...",84
858177,ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha h...,Election,1,0,1,265,299,Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha H...,"[ha, ha, ha, ha, ha, ha, ha, ha, ha, ha, ha, h...",84
1392210,om ledarskap nu good mmer vem som vinner biden...,Election,1,0,0,266,266,Om ledarskap nu bestämmer vem som vinner\nBide...,"[om, ledarskap, nu, good, mmer, vem, som, vinn...",71
620732,om ledarskap nu good mmer vem som vinner biden...,Election,1,0,0,266,266,Om ledarskap nu bestämmer vem som vinner\nBide...,"[om, ledarskap, nu, good, mmer, vem, som, vinn...",71


In [ ]:
# check election dataframe size for 10000
election_df.size

17478050

In [ ]:
election_df.isna().sum()

clean_tweet             0
Topics                  0
Human_generated         0
count_emoji             0
count_profile           0
clean_tweet_Length      0
Tweet_Length            0
Tweet                   0
tokenize_clean_tweet    0
token_length            0
dtype: int64

In [ ]:
election_df[:10000].to_csv('election_df_10k.csv', index=False, encoding='utf-8')

In [ ]:
e10k = pd.read_csv('election_df_10k.csv')
e10k.isna().sum()

clean_tweet             0
Topics                  0
Human_generated         0
count_emoji             0
count_profile           0
clean_tweet_Length      0
Tweet_Length            0
Tweet                   0
tokenize_clean_tweet    0
token_length            0
dtype: int64

In [ ]:
game_df.to_csv('game_df.csv', index=False)

In [ ]:
fifa_df.to_csv('fifa_df.csv', index=False)

In [ ]:
election_df.to_csv('election_df.csv', index=False, encoding='utf-8')

In [ ]:
# example
f = pd.read_csv('fifa_df.csv')
f.iloc[3,7]

'🔥 WL MINT TODAY 🔥\nWhite List Sale 11/20 @ 4PM EST\nCost 003eth\n3 Max mint\n\nPublic Sale 11/21 @ 7AM EST\nCost 005eth\n4 Max mint\n\nTotal Supply 9600\nMint site https//tco/Gkz9cMbCiO\n#NFTsales #NFT #NFTMarketplace #Mint #HotoDoge #WorldCup #WorldCup2022 #Qatar2022 🏆 https//tco/l7JYGdManI'

## Description

In [ ]:
election_df.describe()

,Human_generated,count_emoji,count_username,clean_tweet_Length,Tweet_Length,token_length
count,1747805.00,1747805.00,1747805.00,1747805.00,1747805.00,1747805.00
mean,1.00,0.44,0.66,145.81,166.19,15.67
std,0.00,1.96,1.80,79.29,83.53,9.56
min,1.00,0.00,0.00,1.00,6.00,0.00
25%,1.00,0.00,0.00,79.00,96.00,8.00
50%,1.00,0.00,0.00,136.00,157.00,14.00
75%,1.00,0.00,1.00,218.00,242.00,22.00
max,1.00,137.00,51.00,315.00,987.00,84.00


In [ ]:
election_df.isna().sum()

clean_tweet             0
Topics                  0
Human_generated         0
count_emoji             0
count_username          0
clean_tweet_Length      0
Tweet_Length            0
Tweet                   0
tokenize_clean_tweet    0
token_length            0
dtype: int64

## Gather only English data and randomly select 10k

In [ ]:
# # Gather only English data using detect
# # not select: can't detect ha ha ha as English
# from langdetect import detect

# sentence = "om ledarskap nu good mmer vem som vinner biden..."

# language = detect(sentence)
# print("Detected Language:", language)


In [ ]:
election_pd = pd.read_csv("election_df.csv")
fifa_pd = pd.read_csv("fifa_df.csv")
game_pd = pd.read_csv("game_df.csv")

In [ ]:
game_pd.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,dvr legit look like right matlock 4 new 52 tot...,Game,1,0,0,276,276,my DVR legit looks like this right now\n\nMatl...,"['dvr', 'legit', 'look', 'like', 'right', 'mat...",52
1,1 week till game throne 1 week till game thron...,Game,1,0,0,279,279,1 WEEK TILL GAME OF THRONES\n1 WEEK TILL GAME ...,"['1', 'week', 'till', 'game', 'throne', '1', '...",50
2,number baby ramsay year 1997 1997 12 1998 13 1...,Game,1,0,1,233,265,@clary_olivia The number of babies named Ramsa...,"['number', 'baby', 'ramsay', 'year', '1997', '...",50
3,1 selena 2 dixie chick 3 cat 4 calm 5 friend 6...,Game,1,0,0,258,276,1 Selena \n2 Dixie Chicks \n3 Cats \n4 Calm\n5...,"['1', 'selena', '2', 'dixie', 'chick', '3', 'c...",49
4,william hill odd sit iron throne end game thro...,Game,1,0,0,259,277,William Hill with the odds on who sits on the ...,"['william', 'hill', 'odd', 'sit', 'iron', 'thr...",48


In [ ]:
election_pd.isna().sum()

clean_tweet                 7
Topics                     11
Human_generated            11
count_emoji                24
count_profile              24
clean_tweet_Length         24
Tweet_Length               24
Tweet                      24
tokenize_clean_tweet    63730
token_length            63730
dtype: int64

In [ ]:
game_pd.isna().sum()

clean_tweet             4
Topics                  0
Human_generated         0
count_emoji             0
count_profile           0
clean_tweet_Length      0
Tweet_Length            0
Tweet                   0
tokenize_clean_tweet    0
token_length            0
fasttext_language       4
dtype: int64

In [ ]:
game_pd[game_pd.isnull().any(axis=1)]

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length,fasttext_language
760656,NaN,Game,1,0,0,0,11,,[],0,None
760657,NaN,Game,1,0,0,28,28,ᴳᵃᵐᵉ ᵒᶠ ᵗʰʳᵒⁿᵉˢ ⁱˢ ᵒᵛᵉʳʳᵃᵗᵉᵈ,[],0,None
760658,NaN,Game,1,0,0,1,6,',[],0,None
760659,NaN,Game,1,1,0,31,33,ᴵ’ᵛᵉ ⁿᵉᵛᵉʳ ˢᵉᵉⁿ ᴳᵃᵐᵉ ᵒᶠ ᵀʰʳᵒⁿᵉˢ 🤭,[],0,None


In [ ]:
election_pd.shape

(1811522, 10)

In [ ]:
election_pd = election_pd.dropna()
game_pd = game_pd.dropna()
fifa_pd = fifa_pd.dropna()

In [ ]:
print(election_pd.shape, game_pd.shape, fifa_pd.shape)

(1747785, 11) (760656, 11) (22524, 11)


## select only english

### Langauge detection by fastext (selected)

In [ ]:
model_path = 'lid.176.bin'
model = fasttext.load_model(model_path)
# can't process line with /n, so use clean_tweet
def detect_by_fasttext(text):
    try:
        prediction = model.predict(text)
        language = prediction[0][0].split('__label__')[1]
        return language
#         return detect(x)
    except Exception as e:
        print(text)
        print("Error:", e)
        return None  # or any value to indicate error
#     prediction = model.predict(text)
#     language = prediction[0][0].split('__label__')[1]
#     return language

In [ ]:
game_pd['fasttext_language'] = game_pd['clean_tweet'].apply(lambda x: detect_by_fasttext(x))

In [ ]:
election_pd['fasttext_language'] = election_pd['clean_tweet'].apply(lambda x: detect_by_fasttext(x))

In [ ]:
fifa_pd['fasttext_language'] = fifa_pd['clean_tweet'].apply(lambda x: detect_by_fasttext(x))

In [ ]:
game_df_fasttext = game_pd[game_pd['fasttext_language'] == 'en']

In [ ]:
game_df_fasttext.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length,fasttext_language
0,dvr legit look like right matlock 4 new 52 tot...,Game,1,0,0,276,276,my DVR legit looks like this right now\n\nMatl...,"['dvr', 'legit', 'look', 'like', 'right', 'mat...",52,en
1,1 week till game throne 1 week till game thron...,Game,1,0,0,279,279,1 WEEK TILL GAME OF THRONES\n1 WEEK TILL GAME ...,"['1', 'week', 'till', 'game', 'throne', '1', '...",50,en
3,1 selena 2 dixie chick 3 cat 4 calm 5 friend 6...,Game,1,0,0,258,276,1 Selena \n2 Dixie Chicks \n3 Cats \n4 Calm\n5...,"['1', 'selena', '2', 'dixie', 'chick', '3', 'c...",49,en
4,william hill odd sit iron throne end game thro...,Game,1,0,0,259,277,William Hill with the odds on who sits on the ...,"['william', 'hill', 'odd', 'sit', 'iron', 'thr...",48,en
5,dog 07 2019 ytd tot rtn 886 v tsx 635 max rtn ...,Game,1,0,0,251,251,T-DOGS 07-May-2019\n\nYTD\nTot Rtn 886%\nv T...,"['dog', '07', '2019', 'ytd', 'tot', 'rtn', '88...",48,en


In [ ]:
election_df_fasttext = election_pd[election_pd['fasttext_language'] == 'en']

In [ ]:
fifa_df_fasttext = fifa_pd[fifa_pd['fasttext_language'] == 'en']

In [ ]:
election_df_fasttext.to_csv('election_df_fasttext_en_all.csv', index=False)

In [ ]:
fifa_df_fasttext.to_csv('fifa_df_fasttext_en_all.csv', index=False)

In [ ]:
game_df_fasttext.to_csv('game_df_fasttext_en_all.csv', index=False)

### randomly select

In [ ]:
print(election_df_fasttext.shape, game_df_fasttext.shape, fifa_df_fasttext.shape)

(1318144, 11) (748645, 11) (21719, 11)


In [ ]:
game_pd_fastext_10k = game_df_fasttext.sample(n=10000, random_state=42)  # Replace 100 with the desired number of rows
# Now, sample_df contains a randomly selected subset of rows from the original DataFrame

In [ ]:
election_pd_fastext_10k = election_df_fasttext.sample(n=10000, random_state=42)  # Replace 100 with the desired number of rows
# Now, sample_df contains a randomly selected subset of rows from the original DataFrame

In [ ]:
fifa_pd_fastext_10k = fifa_df_fasttext.sample(n=10000, random_state=42)  # Replace 100 with the desired number of rows
# Now, sample_df contains a randomly selected subset of rows from the original DataFrame

In [ ]:
print(election_pd_fastext_10k.shape, game_pd_fastext_10k.shape, fifa_pd_fastext_10k.shape)

(10000, 11) (100000, 11) (10000, 11)


In [ ]:
game_pd_fastext_10k.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length,fasttext_language
743335,game throne ehhhhh,Game,1,0,2,31,53,@Applebees @Thesixler Just say game of thrones...,"['game', 'throne', 'ehhhhh']",3,en
510260,sure game throne finale look forward day,Game,1,0,0,94,94,Well that sure was a Game of Thrones finale I'...,"['sure', 'game', 'throne', 'finale', 'look', '...",7,en
672165,look forward week url,Game,1,0,0,43,61,Something to look forward to next week \nhttps...,"['look', 'forward', 'week', 'url']",4,en
546218,hot pie survive ht url url,Game,1,0,1,28,75,Hot Pie survived Ht @kimrrenfro https//tco/qUp...,"['hot', 'pie', 'survive', 'ht', 'url', 'url']",6,en
629042,amp ruin game throne gameofthrone,Game,1,0,0,45,45,D&amp;D ruined game of thrones #GameofThrones,"['amp', 'ruin', 'game', 'throne', 'gameofthrone']",5,en


In [ ]:
election_pd_fastext_10k = text_preprocessing(election_pd_fastext_10k)

In [ ]:
election_pd_fastext_10k.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
6494,citizen kill covid19 corona trump 1 million jo...,Election,1.0,0,0,271,271,Citizens killed by #covid19/#Corona/#Trump per...,"[citizen, kill, covid19, corona, trump, 1, mil...",54
9285,mama mnet soty aoty roty dynamite aespa txt st...,Election,1.0,0,0,256,274,mama mnet soty aoty roty dynamite aespa txt st...,"[mama, mnet, soty, aoty, roty, dynamite, aespa...",43
3667,50 100 profit week ggtrade community url eth x...,Election,1.0,1,0,260,296,Make +50-100% profit every week with GGTRADING...,"[50, 100, profit, week, ggtrade, community, ur...",42
3762,iphone 11 12 pro max mini xr xs x 8 7 6 6s plu...,Election,1.0,0,0,250,286,For iPhone 11 12 Pro Max Mini XR XS X 8 7 6 6S...,"[iphone, 11, 12, pro, max, mini, xr, xs, x, 8,...",41
9164,deadly event history 1 675000 1918 flu pandemi...,Election,1.0,0,1,269,303,@ProjectLincoln Deadliest events in US history...,"[deadly, event, history, 1, 675000, 1918, flu,...",39


In [ ]:
game_pd_fastext_10k.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length,fasttext_language
743335,game throne ehhhhh,Game,1,0,2,31,53,@Applebees @Thesixler Just say game of thrones...,"['game', 'throne', 'ehhhhh']",3,en
510260,sure game throne finale look forward day,Game,1,0,0,94,94,Well that sure was a Game of Thrones finale I'...,"['sure', 'game', 'throne', 'finale', 'look', '...",7,en
672165,look forward week url,Game,1,0,0,43,61,Something to look forward to next week \nhttps...,"['look', 'forward', 'week', 'url']",4,en
546218,hot pie survive ht url url,Game,1,0,1,28,75,Hot Pie survived Ht @kimrrenfro https//tco/qUp...,"['hot', 'pie', 'survive', 'ht', 'url', 'url']",6,en
629042,amp ruin game throne gameofthrone,Game,1,0,0,45,45,D&amp;D ruined game of thrones #GameofThrones,"['amp', 'ruin', 'game', 'throne', 'gameofthrone']",5,en


In [ ]:
game_pd_fastext_10k = text_preprocessing(game_pd_fastext_10k)

In [ ]:
fifa_pd_fastext_10k = text_preprocessing(fifa_pd_fastext_10k)

### Check if contain stopwords in clean text

In [ ]:
for i, row in election_pd_fastext_10k.iterrows():
    for w in row['tokenize_clean_tweet']:
        if w in stop_words or w in ENGLISH_STOP_WORDS or w in gensim_stopwords:
            print(row['tokenize_clean_tweet'])

In [ ]:
for i, row in game_pd_fastext_10k.iterrows():
    for w in row['tokenize_clean_tweet']:
        if w in stop_words or w in ENGLISH_STOP_WORDS or w in gensim_stopwords:
            print(row['tokenize_clean_tweet'])

In [ ]:
for i, row in fifa_pd_fastext_10k.iterrows():
    for w in row['tokenize_clean_tweet']:
        if w in stop_words or w in ENGLISH_STOP_WORDS or w in gensim_stopwords:
            print(row['tokenize_clean_tweet'])

In [ ]:
fifa_pd_fastext_10k.describe()

,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,token_length
count,100.0,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1.0,0.620000,0.320000,117.100000,130.260000,10.750000
std,0.0,1.261553,0.897584,60.228605,63.232235,6.059261
min,1.0,0.000000,0.000000,23.000000,23.000000,2.000000
25%,1.0,0.000000,0.000000,69.750000,79.750000,6.000000
50%,1.0,0.000000,0.000000,109.000000,123.500000,9.000000
75%,1.0,1.000000,0.000000,146.250000,165.500000,14.000000
max,1.0,7.000000,7.000000,272.000000,273.000000,33.000000


In [ ]:
game_pd_fastext_10k.describe()

,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,token_length
count,100.0,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1.0,0.020000,0.280000,96.520000,107.760000,9.270000
std,0.0,0.140705,0.604361,57.109938,59.169355,4.737248
min,1.0,0.000000,0.000000,21.000000,23.000000,3.000000
25%,1.0,0.000000,0.000000,55.750000,67.000000,6.000000
50%,1.0,0.000000,0.000000,82.500000,100.000000,8.000000
75%,1.0,0.000000,0.000000,122.250000,133.250000,11.250000
max,1.0,1.000000,3.000000,254.000000,277.000000,24.000000


In [ ]:
election_pd_fastext_10k.describe()

,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,token_length
count,100.0,100.000000,100.000000,100.000000,100.000000,100.000000
mean,1.0,0.880000,0.600000,180.530000,200.970000,16.080000
std,0.0,2.800721,1.015038,68.928224,71.765254,6.398674
min,1.0,0.000000,0.000000,18.000000,36.000000,4.000000
25%,1.0,0.000000,0.000000,119.750000,145.500000,11.000000
50%,1.0,0.000000,0.000000,189.500000,204.500000,16.500000
75%,1.0,0.000000,1.000000,243.250000,269.250000,20.000000
max,1.0,22.000000,5.000000,284.000000,329.000000,32.000000


In [ ]:
game_pd_fastext_10k.isna().sum()

clean_tweet             0
Topics                  0
Human_generated         0
count_emoji             0
count_profile           0
clean_tweet_Length      0
Tweet_Length            0
Tweet                   0
tokenize_clean_tweet    0
token_length            0
language                0
fasttext_language       0
dtype: int64

In [ ]:
game_pd_fastext_10k.to_csv('game_df_fasttext_en_10k.csv', index=False)

In [ ]:
fifa_pd_fastext_10k.to_csv('fifa_df_fasttext_en_10k.csv', index=False)

In [ ]:
election_pd_fastext_10k.to_csv('election_df_fasttext_en_10k.csv', index=False)

In [ ]:
game_pd_fasttext_10k_test = pd.read_csv('game_df_fasttext_en_10k.csv')

In [ ]:
game_pd_fasttext_10k_test.head()

,clean_tweet,Topics,Human_generated,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,sword umbrella fun treasure fan handle carve l...,Game,1,0,0,220,238,The Sword Umbrella is a fun treasure for any G...,"['sword', 'umbrella', 'fun', 'treasure', 'fan'...",24
1,want thank 8 season perfect score game throne ...,Game,1,0,1,254,269,@Djawadi_Ramin I just want to say thank you fo...,"['want', 'thank', '8', 'season', 'perfect', 's...",23
2,arya want equine therapy week game throne emot...,Game,1,0,0,203,221,Arya made me want some equine therapy after la...,"['arya', 'want', 'equine', 'therapy', 'week', ...",21
3,new video post geekandsundry twitch spoiler ki...,Game,1,0,0,165,201,New video posted by GeekandSundry on Twitch SP...,"['new', 'video', 'post', 'geekandsundry', 'twi...",20
4,do not watch game throne becase watch literall...,Game,1,0,2,250,277,@DemonicCasts @nytimesarts I didnt watch Game ...,"['do not', 'watch', 'game', 'throne', 'becase'...",20


In [ ]:
# double check for stopword

In [ ]:
election_user_fasttext_df = pd.read_csv("election_df_fasttext_en_10k.csv")
fifa_user_fasttext_df = pd.read_csv("fifa_df_fasttext_en_10k.csv")
game_user_fasttext_df = pd.read_csv("game_df_fasttext_en_10k.csv")

In [ ]:
tfidf_lst_e = run_tf_idf('election_pd_fastext_10k', election_user_fasttext_df)

election_pd_fastext_10k ['trump', 'url', 'biden', 'joebiden', 'vote', 'election2020', 'donaldtrump', 'win', 'president', 'election', 'bidenharris2020', 'amp', 'elections2020', 'america', 'trump2020', 'like', 'joe', 'people', 'electionday', 'usa', 'maga', 'know', 'think', 'good', 'time', 'biden2020', 'state', 'uselection2020', 'lose', '2020', 'kamalaharris', 'want', 'need', 'covid19', 'year', 'right', 'covid', 'uselection', 'lie', 'day', 'donald', 'let', '2020election', 'democrats', 'electionnight', 'come', 'usaelections2020', 'count', 'bidenharris', 'look']


In [ ]:
tfidf_lst_f = run_tf_idf('fifa_pd_fastext_10k', fifa_user_fasttext_df)

fifa_pd_fastext_10k ['worldcup2022', 'url', 'qatar', 'world', 'cup', 'worldcup', 'fifaworldcup', 'ecuador', 'qatar2022', 'offside', 'qatarworldcup2022', 'open', 'goal', 'fifa', 'game', '2022', 'start', 'football', 'let', 'ceremony', 'win', 'watch', 'team', 'var', 'today', 'good', 'match', 'time', 'worldcupqatar2022', 'fifaworldcup2022', 'begin', 'live', 'like', 'qatecu', 'host', 'england', 'minute', 'day', 'play', 'jungkook', 'corruption', 'look', 'player', 'right', 'kick', 'score', 'fan', 'come', 'tournament', 'amp']


In [ ]:
tfidf_lst_g = run_tf_idf('game_pd_fastext_10k', game_user_fasttext_df)

game_pd_fastext_10k ['game', 'throne', 'url', 'watch', 'episode', 'season', 'like', 'end', 'gameofthrone', 'good', 'tonight', 'time', 'people', 'think', 'start', 'night', 'know', 'day', 'finale', 'fan', 'hbo', 'week', 'spoiler', 'come', 'final', 'tweet', 'new', 'talk', 'want', 'feel', 'character', 'need', 'love', 'wait', 'fuck', 'sunday', 'series', 'thing', 'twitter', 'bad', 'win', 'amp', 'shit', 'lol', 'die', 'today', 'tv', 'way', 'look', 'catch']


## remove URL, count URL, keeps stopwords

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(' ', text)

# Test
# text_with_emojis = "Hello 😊, how are you? 🌟"
# text_without_emojis = remove_emojis(text_with_emojis)
# print(text_without_emojis)  # Output: Hello , how are you?

In [ ]:
# make a text preprocessing function

def text_preprocessing_part3(df):
    """
    return None. Modify in place.
    """
    # =======Removeing=======

    df['Tweet'] = df['Tweet'].apply(lambda x: re.sub('(<.*?>)', '', x))

    # remove punctuation
    df['Tweet'] = df['Tweet'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))

    # remove larger than
    df['Tweet'] = df['Tweet'].str.replace(r'&[gl]t;', '', regex=True)

    # counting @
    df['count_profile'] = df['Tweet'].apply(lambda x: len(re.findall(r'@\w+', x)))

    # Counting emoji
    df['count_emoji'] = df['Tweet'].apply(lambda x: emoji.emoji_count(x))

    # clean tweet removing @ and emoji
    df['clean_tweet'] = df['Tweet'].apply(lambda x: re.sub(r'@\w+', '', remove_emojis(x)))

    # remove whitespace
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: x.strip())

    # temp clean_tweet
    # Define a regular expression to match URLs
    url_pattern = r'https\S+'
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub(url_pattern, 'URL', x))

    # =======================count url ===================================================
    # Count occurrences of "URL" in each row of the 'text_column'
    url_counts = df['clean_tweet'].str.count(r'\bURL\b')

    # Add the counts as a new column in the DataFrame
    df['count_url'] = url_counts

    # delete word URL from text
    # Function to remove URLs from a text
    def remove_urls(text):
        return re.sub(r'\bURL\b', '', text)

    # Remove URLs from 'text_column'
    df['clean_tweet'] = df['clean_tweet'].apply(remove_urls)

    # =======================count url ===================================================

    # only charac
#     df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('[^a-zA-Z"]', ' ', x))
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: re.sub('[^a-zA-Z0-9"]', ' ', x))

    # convert to lower
    df['clean_tweet'] = df['clean_tweet'].apply(lambda x: x.lower())


    # =======count length=======
    df['Tweet_Length'] = df['Tweet'].apply(len)
    df['clean_tweet_Length'] = df['clean_tweet'].apply(len)

    # =======Reorder columns=======
    # Specify the new order of columns
    new_order = ['clean_tweet', 'Topics', 'Human_generated', 'count_url', 'count_emoji', 'count_profile', 'clean_tweet_Length', 'Tweet_Length', 'Tweet']

    # Change the order of columns and
    df = df.reindex(columns=new_order)

    # reset index and return a new copy of dataframe
    df = df.reset_index().iloc[:, 1:].copy()

    # =======Sort and reIndex=======
    ### not doing sort here, because we should sort by count of words. # REMEMBER to deal with https
    # sort by len(clean_tweet)
    # df.sort_values(by='clean_tweet', key=lambda x: x.str.len(), ascending=False, inplace=True)

    return df

In [ ]:
def text_preprocessing_part4(df):
#     # Define a function to map POS tags from Treebank to WordNet
#     def get_wordnet_pos(treebank_tag):
#         if treebank_tag.startswith('J'):
#             return wordnet.ADJ
#         elif treebank_tag.startswith('V'):
#             return wordnet.VERB
#         elif treebank_tag.startswith('N'):
#             return wordnet.NOUN
#         elif treebank_tag.startswith('R'):
#             return wordnet.ADV
#         else:
#             return wordnet.NOUN  # default to noun if POS tag not found

#     def lemmatize_word(word):
#         pos_tag = nltk.pos_tag([word])[0][1]  # Get POS tag of the word
#         pos = get_wordnet_pos(pos_tag)  # Map POS tag to WordNet POS
#         return lemmatizer.lemmatize(word, pos=pos)  # Lemmatize the word
    # =========not use above===can't catch: catch, saw... =====================================================================================

    # Load the English language model
    nlp = spacy.load("en_core_web_sm")

    # Define a function to lemmatize a text
    def lemmatize_text(text):
        doc = nlp(text)
        lemmatized_text = " ".join([token.lemma_ for token in doc])
        return lemmatized_text

    # =======Preprocessing start========================================================================================
    # tokenize
    df['tokenize_clean_tweet'] = df['clean_tweet'].apply(lambda x: text_to_word_sequence(x))

    # remove stop words
#     df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [w for w in x
#                                                                              if w not in stop_words and
#                                                                              not w in ENGLISH_STOP_WORDS and
#                                                                              not w in gensim_stopwords])
    print("lemmatize_text here")
    # lemmatizer: to its original form
    df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [lemmatize_text(token) for token in x])

    # newly add: because "gonna" -> "go to"
    df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [word for w in x for word in text_to_word_sequence(w)])

    # ========================Keep stopwords====================================================
    # remove stop words
#     df['tokenize_clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: [w for w in x
#                                                                              if w not in stop_words and
#                                                                              w not in ENGLISH_STOP_WORDS and
#                                                                              w not in gensim_stopwords])
    # ========================Keep stopwords====================================================

    # transfer back to string clean_tweet
    df['clean_tweet'] = df['tokenize_clean_tweet'].apply(lambda x: " ".join(x))

    # sort by token_length
    df['token_length'] = df['tokenize_clean_tweet'].apply(len)

    df.sort_values(by='token_length', ascending=False, inplace=True)



In [ ]:
def text_preprocessing_del_url_keep_stpW(ori_df):
    df = text_preprocessing_part3(ori_df)
    text_preprocessing_part4(df)
    return df

In [ ]:
# read
game_df = pd.read_csv('game_df_fasttext_en_10k.csv')
fifa_df = pd.read_csv('fifa_df_fasttext_en_10k.csv')
election_df = pd.read_csv('election_df_fasttext_en_10k.csv')

In [ ]:
game_df_machine = pd.read_csv('game_df_machine.csv')
fifa_df_machine = pd.read_csv('fifa_df_machine.csv')
election_df_machine = pd.read_csv('election_df_machine.csv')

In [ ]:
mental_health_df_machine = pd.read_csv('mental_health_df_machine.csv')
airline_df_machine = pd.read_csv('airline_df_machine.csv')

In [ ]:
mental_health_df = pd.read_csv('mental_health_df.csv')
airline_df = pd.read_csv('airline_df.csv')

In [ ]:
fifa_df = text_preprocessing_del_url_keep_stpW(fifa_df)

In [ ]:
fifa_df.to_csv('fifa_df_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
fifa_url = pd.read_csv('fifa_df_en_10k_del_url_w_stpw.csv')
fifa_url.head()

,clean_tweet,Topics,Human_generated,count_url,count_emoji,count_profile,clean_tweet_Length,Tweet_Length,Tweet,tokenize_clean_tweet,token_length
0,worldcup2022 gw1 prediction qat 1 2 ecu sen 0 ...,Fifa,1,0,0,0,279,279,#WorldCup2022 GW1 Predictions\n \n#Qat 1-2 #Ec...,"['worldcup2022', 'gw1', 'prediction', 'qat', '...",67
1,this world cup be corrupt even more now so it ...,Fifa,1,1,0,0,279,300,This World Cup is corrupt even more now so it’...,"['this', 'world', 'cup', 'be', 'corrupt', 'eve...",57
2,hypocrisy it be sad that all this talk here in...,Fifa,1,0,0,0,271,271,Hypocrisy it is sad that all this talk here in...,"['hypocrisy', 'it', 'be', 'sad', 'that', 'all'...",57
3,it only come every 4 year so even though there...,Fifa,1,0,1,0,273,279,It only comes every 4 years so even though the...,"['it', 'only', 'come', 'every', '4', 'year', '...",57
4,the part about not sell alcohol at worldcup202...,Fifa,1,0,0,0,268,268,The part about not selling alcohol at #WorldCu...,"['the', 'part', 'about', 'not', 'sell', 'alcoh...",56


In [ ]:
game_df = text_preprocessing_del_url_keep_stpW(game_df)

lemmatize_text here


In [ ]:
game_df.to_csv('game_df_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
game_df_machine = text_preprocessing_del_url_keep_stpW(game_df_machine)

lemmatize_text here


In [ ]:
game_df_machine.to_csv('game_df_machine_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
election_df_machine = text_preprocessing_del_url_keep_stpW(election_df_machine)

lemmatize_text here


In [ ]:
election_df_machine.to_csv('election_df_machine_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
fifa_df_machine = text_preprocessing_del_url_keep_stpW(fifa_df_machine)

lemmatize_text here


In [ ]:
fifa_df_machine.to_csv('fifa_df_machine_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
airline_df_machine = text_preprocessing_del_url_keep_stpW(airline_df_machine)

lemmatize_text here


In [ ]:
airline_df_machine.to_csv('airline_df_machine_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
mental_health_df_machine = text_preprocessing_del_url_keep_stpW(mental_health_df_machine)

lemmatize_text here


In [ ]:
mental_health_df_machine.to_csv('mental_health_df_machine_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

In [ ]:
airline_df = text_preprocessing_del_url_keep_stpW(airline_df)
airline_df.to_csv('airline_df_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

lemmatize_text here


In [ ]:
mental_health_df = text_preprocessing_del_url_keep_stpW(mental_health_df)
mental_health_df.to_csv('mental_health_en_10k_del_url_w_stpw.csv', index=False, encoding='utf-8')

lemmatize_text here


In [ ]:
import pandas as pd

airline_df = pd.read_csv('mental_health_en_10k_del_url_w_stpw.csv')
airline_df[airline_df['clean_tweet'].str.contains('#url')]
airline_df['count_url'].value_counts()

0     8832
1     1117
2       33
3        7
5        6
16       1
9        1
4        1
13       1
6        1
Name: count_url, dtype: int64